In [74]:
import numpy as np
import random

In [75]:
def initialize(distance_matrix, n):
    min_idx = 0
    solution = []
    
    solution.append(0)
    
    while True:
        i = min_idx
        min_dist = float('inf')
        
        for j in range(n):
            if distance_matrix[i][j] < min_dist and i != j and j not in solution:
                min_idx = j
                min_dist = distance_matrix[i][j]
                
        solution.append(min_idx)
        
        if len(solution) == n:
            break
    
    return solution

In [76]:
def evaluate_solution(solution, distance_matrix, n):
    total_distance = 0
    
    for i in range(n-1):
        current_city = solution[i]
        next_city = solution[i+1]
        
        total_distance += distance_matrix[current_city][next_city]
        
    total_distance += distance_matrix[solution[0]][solution[-1]]
    
    return total_distance

In [77]:
def modify(solution, p1, p2):

    new_solution = solution[:];
    new_solution[p1], new_solution[p2] = new_solution[p2], new_solution[p1]

    return new_solution

In [99]:
def tabu_active(p1, p2, tabu_list, iter):
    if str([p1, p2]) in tabu_list and tabu_list[str([p1,p2])] > iter:
        return True
    return False

In [107]:
def tabu_search(distance_matrix, n, tabu_tenure, num_iters):
    current_solution = initialize(distance_matrix, n)
    current_value = evaluate_solution(current_solution, distance_matrix, n)
    solution = current_solution
    best_value = current_value
    tabu_list = {}
    
    for i in range(num_iters):
        p1 = random.randrange(len(solution))
        p2 = random.randrange(len(solution))

        while p1 == p2:
            p2 = random.randrange(len(solution))
            
        new_solution = modify(current_solution, p1, p2)
        new_value = evaluate_solution(new_solution, distance_matrix, n)
        
        if new_value < current_value and not tabu_active(p1, p2, tabu_list, i):
            current_value = new_value
            current_solution = new_solution
            tabu_list[str([p1, p2])] = i + tabu_tenure
            tabu_list[str([p2, p1])] = i + tabu_tenure
            
            if new_value < best_value:
                best_value = new_value
                solution = new_solution
                
        # clean up the tabu list every 40 iterations       
        if i % 40:
            tabu_list = {k:v for (k, v) in tabu_list.items() if v < i}

    return solution, best_value

In [108]:
def load_data(filename):
    with open(filename, 'r') as f:
        n = int(f.readline())
        distance_matrix = [[int(j) for j in f.readline().split()] for i in range(n)]
    return n, distance_matrix

Testing on the [GR17](https://people.sc.fsu.edu/~jburkardt/datasets/tsp/gr17_d.txt) dataset

In [109]:
n, distance_matrix = load_data('./sample_data/gr17.txt')

In [111]:
tabu_search(distance_matrix, n, 8, 10000)

([0, 3, 12, 6, 7, 5, 16, 13, 14, 2, 10, 9, 1, 4, 8, 11, 15], 2085)